In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
data = pd.read_csv('label_data.csv')
data

,제목,라벨러1,라벨러2,라벨러3,라벨러4,라벨러5,라벨러6
0,"[신년사]손경식 CJ회장_ ""해외사업 공격적 확장_ 성장 가속화""",3,3,3,4,3,3
1,[취업과 창업]신입사원 연차 생기고 공공기관 채용 역대 최대,4,4,5,5,5,4
2,2018년 채용연계형 인턴채용 문 열렸다,4,4,4,4,5,4
3,무술년 채용연계형 인턴채용 스타트,4,3,4,4,5,4
4,그냥 쉬었다 청년_ 30만명 역대 최고치,2,2,1,1,1,1
...,...,...,...,...,...,...,...
7613,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,2,2,2,2,2,2
7614,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,4,4,1,2,2,2
7615,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,2,2,1,2,2,3
7616,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,4,4,4,4,2,3


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7618 entries, 0 to 7617
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      7618 non-null   object
 1   라벨러1    7618 non-null   int64 
 2   라벨러2    7618 non-null   int64 
 3   라벨러3    7618 non-null   int64 
 4   라벨러4    7618 non-null   int64 
 5   라벨러5    7618 non-null   int64 
 6   라벨러6    7618 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 416.7+ KB


In [ ]:
df_mean = data.mean(numeric_only=True, axis='columns')

for i in range(len(df_mean)):
    if df_mean[i] > 3:
        df_mean[i] = 1
    elif df_mean[i] <= 2:
        df_mean[i] = -1
    else:
        df_mean[i] = 0


In [ ]:
df_mean = list(map(int,df_mean))
data['labels'] = df_mean
data

,제목,라벨러1,라벨러2,라벨러3,라벨러4,라벨러5,라벨러6,labels
0,"[신년사]손경식 CJ회장_ ""해외사업 공격적 확장_ 성장 가속화""",3,3,3,4,3,3,1
1,[취업과 창업]신입사원 연차 생기고 공공기관 채용 역대 최대,4,4,5,5,5,4,1
2,2018년 채용연계형 인턴채용 문 열렸다,4,4,4,4,5,4,1
3,무술년 채용연계형 인턴채용 스타트,4,3,4,4,5,4,1
4,그냥 쉬었다 청년_ 30만명 역대 최고치,2,2,1,1,1,1,-1
...,...,...,...,...,...,...,...,...
7613,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,2,2,2,2,2,2,-1
7614,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,4,4,1,2,2,2,0
7615,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,2,2,1,2,2,3,-1
7616,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,4,4,4,4,2,3,1


In [ ]:
data.rename(columns = {'제목':'kor_sentence'},inplace=True)
df = data[['labels','kor_sentence']]
df

,labels,kor_sentence
0,1,"[신년사]손경식 CJ회장_ ""해외사업 공격적 확장_ 성장 가속화"""
1,1,[취업과 창업]신입사원 연차 생기고 공공기관 채용 역대 최대
2,1,2018년 채용연계형 인턴채용 문 열렸다
3,1,무술년 채용연계형 인턴채용 스타트
4,-1,그냥 쉬었다 청년_ 30만명 역대 최고치
...,...,...
7613,-1,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배
7614,0,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소
7615,-1,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사
7616,1,토스_ 고객 상담직 신입 및 경력 30명 공개 채용


In [ ]:
duplicate = df[df.duplicated('kor_sentence')]
duplicate

,labels,kor_sentence
2619,0,정부 취약층 일자리 10명중 7명꼴 노인
2632,1,대기업 영업익-투자 30% 줄어도 고용은 1년새 1만5000명 늘어
2639,-1,정부 최저임금 올라 고용감소 첫 인정
2657,-1,중형 조선사 5곳 직원수 8년새 반토막
2666,0,[J노믹스 2년]시련의 일자리 대통령 고용 절벽 부른 최저임금
2674,1,신용보증기금_ 11월말까지 '온라인 채용박람회' 개최
2685,1,근로시간 준수 가족친화기업_ 일자리 창출-경영실적 좋아
2696,-1,대졸실업자 2년만에 또 사상최고 청년고용 개선 맞나
2698,1,대기업_ 수익 투자 1년새 30%대 급감해도 고용은 1만5천명 늘려
2713,0,"국회예정처 ""경제성장률 1%p 오르면 취업자 증가율 0.41%p """


In [ ]:
df.drop_duplicates(['kor_sentence'], inplace=True)
len(df)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


7595

In [ ]:
print(df.groupby('labels').size().reset_index(name='count'))
print(f'중립의 비율 = {round(df["labels"].value_counts()[0]/len(df) * 100,3)}%')
print(f'긍정의 비율 = {round(df["labels"].value_counts()[1]/len(df) * 100,3)}%')
print(f'부정의 비율 = {round(df["labels"].value_counts()[-1]/len(df) * 100,3)}%')

   labels  count
0      -1   1816
1       0   2038
2       1   3741
중립의 비율 = 26.833%
긍정의 비율 = 49.256%
부정의 비율 = 23.91%


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 345 kB/s 
     |████████████████████████████████| 465 kB 43.4 MB/s 


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
train_data = df.sample(frac=0.7)
# test_data = df-train_data 아래 코드
test_data = pd.merge(train_data, df, how='outer',indicator = True).query('_merge == "right_only"').drop(columns=['_merge'])

In [ ]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [ ]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도','을', '를', '으로', '자', '에', '와', '한', '하다']

In [ ]:
X_train = []
for sentence in train_data['kor_sentence']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_train.append(temp_X)
  
X_test = []
for sentence in test_data['kor_sentence']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

In [ ]:
print(X_train[:5])
print(X_test[:5])

[['실업', '급여', '보험료', '율', '근로자', '매년', '보험료', '7만원', '더', '내다'], ['[', '목멱', '칼럼', ']', '내년', '일자리', '빨갛다', '불', '원인', '부터', '바로', '알다'], ['SOHO', '무역', '이모티콘', '디자인', '나다', '커리어', '우먼'], ['고용', '노동', '행정', '15', '개', '조사', '과제', '선정', '_', '본격', '조사', '개시'], ['JB', '금융', '그룹', '_', '올해', '채용', '규모', '전', '년', '확대', '전북', '광주', '_', '하반기', '150', '명', '채용']]
[['채용', '공고', '안', '나오다', '직장', '인들', '이직', '줄다'], ['법', '절차', '무시', '개입', "'", '갈등', '활주로', "'", '내', '달리', '인국공', '사태'], ['인국공', '사태', '_', '취준생', '무관', '분노', '기름', '부다', '해명'], ['공공', '분야', '정규직', '전환', '선언', '3년', '목표', '90%', '이루다', '곳곳', '서', '파열음'], ['인천', '시', '_', '2022년', '까지', '노인', '일자리', '1만', '8천', '개', '창', '추다']]


In [ ]:
from keras.preprocessing.text import Tokenizer
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print("문장의 최대 길이 : ", max(len(l) for l in X_train))
print("문장의 평균 길이 : ", sum(map(len, X_train))/ len(X_train))

문장의 최대 길이 :  25
문장의 평균 길이 :  11.491723100075244


In [ ]:
print("문장의 최대 길이 : ", max(len(l) for l in X_test))
print("문장의 평균 길이 : ", sum(map(len, X_test))/ len(X_test))


문장의 최대 길이 :  24
문장의 평균 길이 :  10.903905221588415


In [ ]:
y_train = []
y_test = []

for i in range(len(train_data['labels'])):
  if train_data['labels'].iloc[i] == 1:  # 긍정(제일 뒤)
    y_train.append([0, 0, 1])
  elif train_data['labels'].iloc[i] == 0:  # 중립(중간)
    y_train.append([0, 1, 0])
  elif train_data['labels'].iloc[i] == -1:  # 부정(제일 앞)
    y_train.append([1, 0, 0])

for i in range(len(test_data['labels'])):
  if test_data['labels'].iloc[i] == 1:
    y_test.append([0, 0, 1])
  elif test_data['labels'].iloc[i] == 0:
    y_test.append([0, 1, 0])
  elif test_data['labels'].iloc[i] == -1:
    y_test.append([1, 0, 0])

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences
max_len = 25 # 전체 데이터의 길이를 28로 맞춘다

X_train = pad_sequences(X_train, maxlen=max_len)  # 
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=20, validation_split=0.2)

Epoch 1/10
213/213 [==============================] - 14s 52ms/step - loss: 0.7894 - accuracy: 0.6258 - val_loss: 0.8180 - val_accuracy: 0.6090
Epoch 2/10
213/213 [==============================] - 10s 48ms/step - loss: 0.5559 - accuracy: 0.7686 - val_loss: 0.6114 - val_accuracy: 0.7265
Epoch 3/10
213/213 [==============================] - 11s 49ms/step - loss: 0.4438 - accuracy: 0.8255 - val_loss: 0.5849 - val_accuracy: 0.7378
Epoch 4/10
213/213 [==============================] - 12s 55ms/step - loss: 0.3642 - accuracy: 0.8612 - val_loss: 0.6408 - val_accuracy: 0.7293
Epoch 5/10
213/213 [==============================] - 10s 49ms/step - loss: 0.3081 - accuracy: 0.8928 - val_loss: 0.6561 - val_accuracy: 0.7293
Epoch 6/10
213/213 [==============================] - 10s 48ms/step - loss: 0.2599 - accuracy: 0.9076 - val_loss: 0.6600 - val_accuracy: 0.7406
Epoch 7/10
213/213 [==============================] - 10s 48ms/step - loss: 0.2185 - accuracy: 0.9250 - val_loss: 0.7473 - val_accuracy:

In [ ]:
predict = model.predict(X_test)
predict_labels = np.argmax(predict, axis=1)
original_labels = np.argmax(y_test, axis=1)

72/72 [==============================] - 2s 20ms/step


In [ ]:
cnt = 0
for i in range(len(test_data)):  # 0:부정, 1:중립, 2:긍정
    print("원래 라벨 : ", original_labels[i], " 예측한 라벨 : ", predict_labels[i],"  문장 : ", test_data['kor_sentence'].iloc[i])
    if original_labels[i] == predict_labels[i]:
        cnt += 1
accuracy = cnt / len(test_data)
print(accuracy)

원래 라벨 :  1  예측한 라벨 :  0   문장 :  채용 공고 안 나와서 직장인들 이직도 줄었다
원래 라벨 :  1  예측한 라벨 :  1   문장 :  법 절차 무시한 개입에 '갈등의 활주로' 내달리는 인국공 사태
원래 라벨 :  1  예측한 라벨 :  1   문장 :  인국공 사태_ 취준생과 무관 분노에 기름 부은 해명
원래 라벨 :  1  예측한 라벨 :  1   문장 :  공공분야 정규직 전환 선언 3년 목표 90% 이뤘지만 곳곳서 파열음
원래 라벨 :  2  예측한 라벨 :  2   문장 :  인천시_ 2022년까지 노인일자리 1만8천개 창출
원래 라벨 :  1  예측한 라벨 :  1   문장 :  코로나19에 행사대행업 매출 75% "행사대행업 특별고용지원업종 추가 필요"
원래 라벨 :  2  예측한 라벨 :  2   문장 :  [다시 일어서는 동남권 경제] 부산시_ 히든챔피언 강소기업 육성..민간서 일자리 창출 활발
원래 라벨 :  2  예측한 라벨 :  0   문장 :  내년부터 대리운전사 보험설계사도 실업급여
원래 라벨 :  2  예측한 라벨 :  0   문장 :  [종합] News Briefing
원래 라벨 :  1  예측한 라벨 :  2   문장 :  특고 프리랜서 등에 150만원 고용안정지원금 신청 90만건 돌파
원래 라벨 :  1  예측한 라벨 :  1   문장 :   아시아 은행 일자리 40% 10년내 디지털 자동화로 대체 
원래 라벨 :  0  예측한 라벨 :  0   문장 :  삐끗하면 실업 못 벗어나 2010년 이후 노동이동 둔화
원래 라벨 :  2  예측한 라벨 :  2   문장 :  "2022년까지 혁신형 소상공인업체 1만5천개 육성"
원래 라벨 :  0  예측한 라벨 :  0   문장 :  취업자 11년만에 최대 감소 코로나 일시휴직자도 폭증
원래 라벨 :  0  예측한 라벨 :  0   문장 :  '일자리' 외친 정부의 오명 어둡기만 한 고용 전망
원래 라벨 :  1  예측한 라벨 :  0   문장 :  [시장 못 이

In [ ]:
print(train_data.groupby('labels').size().reset_index(name='count'))
print(f'중립의 비율 = {round(train_data["labels"].value_counts()[0]/len(train_data) * 100,3)}%')
print(f'긍정의 비율 = {round(train_data["labels"].value_counts()[1]/len(train_data) * 100,3)}%')
print(f'부정의 비율 = {round(train_data["labels"].value_counts()[-1]/len(train_data) * 100,3)}%')
print(test_data.groupby('labels').size().reset_index(name='count'))
print(f'중립의 비율 = {round(test_data["labels"].value_counts()[0]/len(test_data) * 100,3)}%')
print(f'긍정의 비율 = {round(test_data["labels"].value_counts()[1]/len(test_data) * 100,3)}%')
print(f'부정의 비율 = {round(test_data["labels"].value_counts()[-1]/len(test_data) * 100,3)}%')

   labels  count
0      -1   1267
1       0   1438
2       1   2611
중립의 비율 = 27.05%
긍정의 비율 = 49.116%
부정의 비율 = 23.834%
   labels  count
0      -1    549
1       0    600
2       1   1130
중립의 비율 = 26.327%
긍정의 비율 = 49.583%
부정의 비율 = 24.09%


In [ ]:
# 실제 크롤링한 데이터

In [ ]:
data = pd.read_csv('news_main2.csv')
data

,news title,content
0,라임 때 처음 등장한 'CB 공장'…당국 무관심이 피해 키웠다,"""전환사채(CB) 자체가 나쁜 게 아닙니다. 자본 시장을 좀먹는 'CB공장'이 수백..."
1,금융당국 무관심이 무자본 M&A 세력 키웠다,"""전환사채(CB) 자체가 나쁜 게 아닙니다. 자본 시장을 좀먹는 'CB공장'이 수백..."
2,"거래소가 부실기업 퇴출 머뭇거리자…""먹잇감 늘었다"" 남몰래 웃는 검은세력",▶마켓인사이트 12월 13일 오후 5시17분 올해 코스닥시장에서 17개사가 상장폐지...
3,“젊고 기민한 APAC이 전 세계 게임산업 성장 주도할 것”,사미 키질바쉬 구글 액셀러레이터 글로벌 총괄이 13일 라운드테이블을 통해 아태지역의...
4,스타트업의 성공적인 엑시트를 위한 지침서,"[머니투데이 김상희 기자] [[서평]스타트업, 아름다운 성공] 유니콘의 시대다..."
5,"구글 인디게임 총괄 ""5G 인프라 갖춘 한국, 클라우드 게임 성공 여건""",구글 인디 게임즈 프로그램 구글이 아시아·태평양(APAC)을 미래 게임 시장에서 가...
6,[기고] 미드 ‘와이어’로 보는 마약과의 술래잡기,권초희 동국대 와이즈캠퍼스 재학 미국 드라마 <와이어>는 볼티모어에서 일어나는 범죄...
7,"닥공축구 아르헨, 좀비축구 크로아티아","아르헨티나, 크로아티아 아르헨티나와 크로아티아는 각각 월드컵에 한이 많다. 두 나라..."


In [ ]:
contents = []
count = 0
for i in data['content']:
    contents.append(i.split('.'))
for n in range(len(contents)):
    for sentence in contents[n]:
        print(sentence)

"전환사채(CB) 자체가 나쁜 게 아닙니다
 자본 시장을 좀먹는 'CB공장'이 수백개 양산된 게 문제죠
 라임 사태 때 드러난 머니게임 수법을 금융당국이 방치한 결과입니다
"  코스닥시장 무자본 M&A의 첫 제도권 ‘돈줄’은 라임 펀드였다
 라임자산운용은 증권사와 손잡고 라임 펀드에 레버리지를 일으킨 뒤 ‘전환사채(CB) 쇼핑’을 하면서 기업사냥꾼에게 수조원을 대줬다
 2019년 7월 라임 사태가 수면 위로 올라온 뒤 CB 공장을 둘러싼 머니게임이 세상에 모습을 드러냈다
 금융당국은 CB 공장의 폐해를 인지하고도 방치했다
 라임 펀드 투자자 보상과 금융회사 최고경영자(CEO) 제재에만 몰두했다
  그 여파는 심각하다
 코스닥 적자 좀비기업이 갈수록 늘고 있다
 13일 한국거래소에 따르면 코스닥시장의 12월 결산법인 1328곳 가운데 396곳(30%)이 지난해 영업손실을 냈다
 적자 규모는 3조797억원에 달한다
 2015년과 비교해 적자 기업 비중은 9%포인트 늘었고 적자 규모는 세 배 가까이 된다
 올해는 더 늘어날 것으로 보인다
  머니게임이 지능화하면서 투자자 피해는 날로 커지고 있다
 금융당국이 일부 불공정거래에만 몰두할 뿐 투자자 보호를 위한 시장 관리에는 손을 놓고 있다는 지적이 나온다
  한 자본시장 전문 변호사는 “이제라도 금융당국이 개인투자자 보호를 위한 합동 태스크포스(TF)를 구성해 대수술에 나서야 한다”고 말했다
   라임과 기업사냥꾼 의기투합…좀비기업 CB 수조원치 담아 금감원 등이 폐해 인지했지만 무자본 M&A에 큰 신경 안써무자본 인수합병(M&A)을 일삼는 기업사냥꾼들이 전환사채(CB) ‘마법’에 눈 뜬 건 5년 전쯤이다
 한국형 헤지펀드 시장이 활짝 열렸을 때였다
 금융위원회가 모험자본 육성을 외치자 헤지펀드들은 혁신기업 CB 투자로 화답했다
 이때 코스닥 한계기업 CB가 의외로 돈이 된다는 걸 눈치챈 곳이 있었다
 라임자산운용이었다
 라임 일부 경영진은 기업사냥꾼들과 의기투합했다
  라임 펀드는 ‘CB 공장’에 전방위로 돈을 대기

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences

In [ ]:
predict_final = {}
for n in range(len(contents)):
    input_data = []
    for sentence in contents[n]:
        temp_input = []
        temp_input = okt.morphs(sentence, stem=True) # 토큰화
        temp_input = [word for word in temp_input if not word in stopwords] # 불용어 제거
        input_data.append(temp_input)
    
    max_words = 35000
    tokenizer = Tokenizer(num_words = max_words)  # 단어를 크기가 35000인 벡터로 저장, 35000개 이상의 단어가 있다면 해싱 충돌이 발생하고 인코딩의 정확도 감소
    tokenizer.fit_on_texts(input_data)
    input_data = tokenizer.texts_to_sequences(input_data)
    
    max_len = max(len(l) for l in input_data)
    input_data = pad_sequences(input_data, maxlen=max_len)

    predict = model.predict(input_data)
    predict_labels = np.argmax(predict, axis=1)

    predict_final[f'기사{n}'] = sum(predict_labels)
    for i in range(len(contents[n])):  # 0:부정, 1:중립, 2:긍정
        print("예측한 라벨 : ", predict_labels[i],"  기사 문장: ", contents[n][i])

3/3 [==============================] - 0s 23ms/step
예측한 라벨 :  2   기사 문장:  "전환사채(CB) 자체가 나쁜 게 아닙니다
예측한 라벨 :  1   기사 문장:   자본 시장을 좀먹는 'CB공장'이 수백개 양산된 게 문제죠
예측한 라벨 :  1   기사 문장:   라임 사태 때 드러난 머니게임 수법을 금융당국이 방치한 결과입니다
예측한 라벨 :  1   기사 문장:  "  코스닥시장 무자본 M&A의 첫 제도권 ‘돈줄’은 라임 펀드였다
예측한 라벨 :  0   기사 문장:   라임자산운용은 증권사와 손잡고 라임 펀드에 레버리지를 일으킨 뒤 ‘전환사채(CB) 쇼핑’을 하면서 기업사냥꾼에게 수조원을 대줬다
예측한 라벨 :  0   기사 문장:   2019년 7월 라임 사태가 수면 위로 올라온 뒤 CB 공장을 둘러싼 머니게임이 세상에 모습을 드러냈다
예측한 라벨 :  1   기사 문장:   금융당국은 CB 공장의 폐해를 인지하고도 방치했다
예측한 라벨 :  1   기사 문장:   라임 펀드 투자자 보상과 금융회사 최고경영자(CEO) 제재에만 몰두했다
예측한 라벨 :  2   기사 문장:    그 여파는 심각하다
예측한 라벨 :  2   기사 문장:   코스닥 적자 좀비기업이 갈수록 늘고 있다
예측한 라벨 :  2   기사 문장:   13일 한국거래소에 따르면 코스닥시장의 12월 결산법인 1328곳 가운데 396곳(30%)이 지난해 영업손실을 냈다
예측한 라벨 :  2   기사 문장:   적자 규모는 3조797억원에 달한다
예측한 라벨 :  1   기사 문장:   2015년과 비교해 적자 기업 비중은 9%포인트 늘었고 적자 규모는 세 배 가까이 된다
예측한 라벨 :  2   기사 문장:   올해는 더 늘어날 것으로 보인다
예측한 라벨 :  1   기사 문장:    머니게임이 지능화하면서 투자자 피해는 날로 커지고 있다
예측한 라벨 :  2   기사 문장:   금융당국이 일부 불공정거래에만 몰두할 뿐 투자자 보호를 위한